In [2]:
import sys
sys.path.append('../')
sys.path.append('../../')
 
from db_data_downloader.downloader_base import DownloaderBase
from db_factor_prebuilder.utils.expression_excutor import AlphaExpressionExcutor
import json
import sqlite3
import database_config as db_config

db_conn = sqlite3.connect('../../hh_quant_auto.db')
db_downloader = DownloaderBase(db_conn, db_config)
exp_excutor = AlphaExpressionExcutor()

stock_base = db_downloader._download_history_base_info('sh.600011', '2013-01-01', '2014-01-01')
stock_base['vwap'] = stock_base[['open', 'high', 'low', 'close']].mean(axis=1)
stock_base['ret'] = stock_base['close'].pct_change()

In [4]:
# stock_base.head()

In [5]:
alpha_factor_dict = json.loads(open("../factor_lib/alpha_191.json", "r").read())
dataframe = stock_base[["code", "datetime"]]

error_expression_list = []
for alpha_name, alpha_expression in alpha_factor_dict.items():
    try:
        dataframe[alpha_name] = exp_excutor.excute(stock_base, alpha_expression)
    except Exception as e:
        print(f"{alpha_name}: {alpha_expression}")
        print(e)
        error_expression_list.append(f"{alpha_name}: {alpha_expression}")
        # break

alpha_191_3: sum((close=shift(close,1)?0:close-(close>shift(close,1)?min(low,shift(close,1)):max(high,shift(close,1)))),6)
invalid syntax (<string>, line 1)
alpha_191_4: ((((sum(close,8)/8)+std(close,8))<(sum(close,2)/2))?(-1*1):(((sum(close,2)/2)<((sum(close,8)/8)-std(close,8)))?1:(((1<(volume/mean(volume,20)))||((volume/mean(volume,20))==1))?1:(-1*1))))
invalid syntax (<string>, line 1)
alpha_191_10: (rank(max(((ret<0)?std(ret,20):close)^2),5))
invalid syntax (<string>, line 1)
alpha_191_11: sum(((close-low)-(high-close))./(high-low).*volume,6)
invalid syntax (<string>, line 1)
alpha_191_19: (close<shift(close,5)?(close-shift(close,5))/shift(close,5):(close=shift(close,5)?0:(close-shift(close,5))/close))
invalid syntax (<string>, line 1)
alpha_191_21: regbeta(mean(close,6),sequence(6))
name 'regbeta' is not defined
alpha_191_22: smean(((close-mean(close,6))/mean(close,6)-shift((close-mean(close,6))/mean(close,6),3)),12,1)
name 'smean' is not defined
alpha_191_23: sma((close>shift(clo

In [6]:
len(error_expression_list)

66

In [4]:
# error_expression_list

In [10]:
# dataframe.tail()

In [4]:
print(dataframe.tail().isna().sum().sum())

0
